This is my first analysis in the kaggle , also it's own first attainment that opened in public as analyst . 
So I am nothing but layman as analyst , besides my method may wrong or sort of not right way to analyze .
that's why if someone sees this one , I wish that guys write a comment so that I improve .

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import seaborn as sns

from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestClassifier

import sys

import warnings

warnings.simplefilter('ignore')

In [ ]:
#import data
df = pd.read_csv('../input/dummy-advertising-and-sales-data/Dummy Data HSS.csv')
df.head()

# **Data exploration**

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

I fill null data with each mean value .

In [ ]:
df['TV'].fillna(df['TV'].mean(),inplace=True)
df['Radio'].fillna(df['Radio'].mean(),inplace=True)
df['Social Media'].fillna(df['Social Media'].mean(),inplace=True)
df['Sales'].fillna(df['Sales'].mean(),inplace=True)

In [ ]:
plt.pie(df['Influencer'].value_counts(),
   labels=df['Influencer'].unique(), counterclock=False, startangle=90,
   autopct='%1.1f%%', pctdistance=0.7)

According to pie chart , propotions of influencer are equivalent propotion.

In [ ]:
categorical_columns = ['TV', 'Radio', 'Social Media']

RP=plt.figure(figsize=(10,3))
for i, feature in enumerate(categorical_columns):
    r=RP.add_subplot(1,3,i+1)
    #plt.hist(np.log(df[feature]))
    plt.hist(df[feature])
    r.set_title(feature+" Histogram Plot",color='DarkRed')
RP.tight_layout()  

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(15,5))
for idx,cat_col in enumerate(categorical_columns):
    r=fig.add_subplot(1,3,idx+1)
    grid = sns.scatterplot(data=df, x=cat_col, y='Sales', hue="Influencer")
    grid.set_xticklabels('') 
    grid.set_yticklabels('') 
    
    r.set_title(cat_col,color='DarkRed')
fig.tight_layout(rect=[0,0,1,0.96])  

In [ ]:
plt.figure()
df.groupby(["Influencer"])['Sales'].mean().sort_values(ascending=False).plot(kind="bar",figsize=(10, 6))

plt.xticks(rotation=90)
plt.grid(True)
plt.show()

**Train - Test Split**

In [ ]:
df_enc=pd.get_dummies(df,drop_first=True)

X=df_enc.drop(['Sales'],axis=1)
y = df_enc['Sales']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

**Model Creation**
I used following models at the same time by for loop to compare these accuracy .

* LinearRegression
* DecisionTreeRegressor
* RandomForestRegressor
* XGBRegressor
* Support Vector Regression

In [ ]:
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold
from xgboost import XGBRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

models=[('LR',LinearRegression()),('DT',DecisionTreeRegressor()),('RF',RandomForestRegressor()),('XGB',XGBRegressor())
        ,('SVR',SVR())]
scores=[]

for name,model in models:
    model.fit(X_train,y_train)
    preds=model.predict(X_test)
    score=explained_variance_score(preds,y_test)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    scores.append([name,model,mae,r2,score])

In [ ]:
scores_df=pd.DataFrame(scores,columns=['Name','Model','MAE','R2','Score'])
scores_df.sort_values('Score',ascending=False)

scores_df.set_index('Name', drop=True,inplace=True)
scores_df